In [7]:
#!pip install earthengine-api geemap
#!pip install pydrive

In [31]:
import ee
import pandas as pd

# Autenticar y inicializar Earth Engine
ee.Authenticate()
ee.Initialize()

# Definir las cuatro coordenadas (por ejemplo, como latitud y longitud)
punto1 = [39.446215, -0.386607]
punto2 = [39.445682, -0.369052]
punto3 = [39.436768, -0.370158]
punto4 = [39.438423, -0.389371]

# Crear una región poligonal usando los cuatro puntos
roi = ee.Geometry.Polygon([punto1, punto2, punto3, punto4])

# Cargar la colección de imágenes Landsat 8 Collection 2, Level 2
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
  .filterBounds(roi) \
  .filterDate('1995-01-01', '2024-07-30') \
  .filter(ee.Filter.lt('CLOUD_COVER', 20))

# Función para calcular el NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return image.addBands(ndvi)

# Función para calcular LST (Land Surface Temperature)
def calculate_lst(image):
    thermal_bands = image.select('ST_B10').multiply(0.00341802).add(149).subtract(273.15).rename('LST')
    return image.addBands(thermal_bands)

# Aplicar las funciones de NDVI y LST a la colección
landsat8_daily = landsat8.map(calculate_ndvi).map(calculate_lst)

# Cargar la colección de datos meteorológicos de ERA5
era5_daily = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
  .filterBounds(roi) \
  .filterDate('1995-01-01', '2024-07-30')

# Definir los puntos para muestrear (usar los cuatro puntos específicos)
points = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point(punto1), {}),
    ee.Feature(ee.Geometry.Point(punto2), {}),
    ee.Feature(ee.Geometry.Point(punto3), {}),
    ee.Feature(ee.Geometry.Point(punto4), {})
])

# Función para extraer datos diarios
def extract_daily_data(image):
    date = image.date().format('YYYY-MM-dd').getInfo()  # Obtener la fecha de la imagen
    era5_image = era5_daily.filterDate(date).first()  # Obtener la imagen de ERA5 correspondiente al día
    combined_daily = image.addBands(era5_image)
    samples = combined_daily.sampleRegions(
        collection=points,
        scale=1000,
        geometries=True
    )
    sample_dict = samples.getInfo()
    features = sample_dict['features']
    data = [feature['properties'] for feature in features]
    df = pd.DataFrame(data)
    df['date'] = date  # Añadir la fecha al DataFrame
    return df

# Aplicar la extracción a todas las imágenes diarias de Landsat 8
landsat_list = landsat8_daily.toList(landsat8_daily.size())
dfs = []
for i in range(landsat_list.size().getInfo()):
    image = ee.Image(landsat_list.get(i))
    df = extract_daily_data(image)
    dfs.append(df)

# Concatenar todos los DataFrames en uno solo
full_df = pd.concat(dfs, ignore_index=True)

# Guardar el DataFrame como un archivo CSV local
full_df.to_csv('Valencia_Stats_2023_local_daily_values_four_points.csv', index=False)

print("Datos diarios guardados en 'Valencia_Stats_2023_local_daily_values_four_points.csv'")

Datos diarios guardados en 'Valencia_Stats_2023_local_daily_values_four_points.csv'


In [32]:
full_df

,LST,NDVI,QA_PIXEL,QA_RADSAT,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,...,total_evaporation_hourly,total_precipitation,total_precipitation_hourly,u_component_of_wind_10m,v_component_of_wind_10m,volumetric_soil_water_layer_1,volumetric_soil_water_layer_2,volumetric_soil_water_layer_3,volumetric_soil_water_layer_4,date
0,39.870526,0.184691,21824,0,8825,9624,12305,14188,20616,21745,...,-3.041234e-05,3.687933e-03,8.642673e-07,-0.352798,2.854431,0.300201,0.198776,0.150909,0.150497,2013-04-14
1,42.273394,0.204838,23888,0,8768,9252,11458,13641,20669,21146,...,-3.041234e-05,3.687933e-03,8.642673e-07,-0.352798,2.854431,0.300201,0.198776,0.150909,0.150497,2013-04-14
2,29.824965,0.119330,22280,0,14702,15368,17579,19240,24454,23312,...,-3.041234e-05,3.687933e-03,8.642673e-07,-0.352798,2.854431,0.300201,0.198776,0.150909,0.150497,2013-04-14
3,42.444295,0.195788,21824,0,8689,9380,11772,13555,20155,21110,...,-3.041234e-05,3.687933e-03,8.642673e-07,-0.352798,2.854431,0.300201,0.198776,0.150909,0.150497,2013-04-14
4,35.122896,0.180604,22280,0,9086,10110,13085,15394,22180,22678,...,-2.637506e-06,3.950935e-05,3.387697e-09,-0.812973,3.703690,0.115341,0.165543,0.151169,0.150497,2013-04-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,42.656212,0.161510,21824,0,8730,9522,11700,13942,19313,20826,...,-7.152557e-07,8.583069e-07,0.000000e+00,-0.706802,4.679932,0.109375,0.149750,0.155243,0.154144,2024-05-30
332,39.306552,0.120429,21824,0,8858,9877,12263,14779,18826,21111,...,-3.166497e-07,2.671247e-05,1.978378e-09,-0.300278,3.823502,0.109329,0.146500,0.153397,0.153915,2024-07-01
333,39.938886,0.124166,21824,0,8992,9745,11849,14841,19049,21457,...,-3.166497e-07,2.671247e-05,1.978378e-09,-0.300278,3.823502,0.109329,0.146500,0.153397,0.153915,2024-07-01
334,40.807063,0.121708,21824,0,9070,9824,11990,15057,19230,21636,...,-3.166497e-07,2.671247e-05,1.978378e-09,-0.300278,3.823502,0.109329,0.146500,0.153397,0.153915,2024-07-01


In [33]:
full_df.to_csv('Valencia_Stats_2013-2024_local_daily_values_four_points.csv', index=False)